In [93]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
import scipy.stats as st
from scipy.stats import linregress

In [94]:
# Output File (CSV)
pyre_data_path = "Resources/California_Fire_Incidents.csv"

In [95]:
pyre_data = pd.read_csv(pyre_data_path)
pyre_data.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,Started,Time.1,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,8/17/13,15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,5/30/13,15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,7/15/13,13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,8/10/13,16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,5/2/13,07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [96]:
pyre_df = pyre_data[["AcresBurned", "AirTankers", "ArchiveYear", "Counties", "CountyIds","CrewsInvolved", "Dozers", "Engines", "Extinguished", "Days Burned", "Fatalities", "Helicopters", "Injuries", "Latitude", "Location", "MajorIncident", "Name", "PersonnelInvolved", "Started", "StructuresDamaged", "StructuresDestroyed", "StructuresEvacuated", "StructuresThreatened", "UniqueId", "WaterTenders" 
]]
pyre_df.head()

,AcresBurned,AirTankers,ArchiveYear,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,MajorIncident,Name,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders
0,257314.0,NaN,2013,Tuolumne,55,NaN,NaN,NaN,9/6/13,20,...,False,Rim Fire,NaN,8/17/13,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,NaN
1,30274.0,NaN,2013,Los Angeles,19,NaN,NaN,NaN,6/8/13,9,...,False,Powerhouse Fire,NaN,5/30/13,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,NaN
2,27531.0,NaN,2013,Riverside,33,NaN,NaN,NaN,7/30/13,15,...,False,Mountain Fire,NaN,7/15/13,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,NaN
3,27440.0,NaN,2013,Placer,31,NaN,NaN,NaN,8/30/13,20,...,False,American Fire,NaN,8/10/13,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,NaN
4,24251.0,NaN,2013,Ventura,56,47.0,8.0,117.0,5/11/13,9,...,True,Springs Fire,2167.0,5/2/13,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,11.0


In [97]:
counties = pyre_df["Counties"].value_counts()
counties

Riverside          146
San Diego           89
Butte               66
San Luis Obispo     64
Shasta              64
Kern                62
Fresno              57
Siskiyou            57
San Bernardino      53
Tehama              51
Lake                49
Los Angeles         46
Monterey            45
Santa Clara         39
El Dorado           37
Lassen              36
Madera              36
Tulare              35
Mariposa            35
Alameda             32
Modoc               31
Ventura             30
Santa Barbara       29
Mendocino           28
Contra Costa        27
Humboldt            25
Napa                25
Calaveras           22
Tuolumne            22
Trinity             21
Stanislaus          20
San Benito          20
Solano              19
Sonoma              18
Placer              17
Nevada              17
Merced              16
Yuba                14
Amador              13
Inyo                12
Yolo                12
Sacramento          11
Plumas              11
Mono       

In [98]:
unique_id = pyre_df["UniqueId"].value_counts()
unique_id.head(30)

90b0daf7-5d84-42d9-bb35-bb78d4faf950    4
5ce45b6c-81b5-422c-9078-8c68dd872c8a    4
af30fe23-6cfd-4fd6-b2d9-91648eeae814    3
4d07ccc3-32be-4a1d-acac-18333884c97d    2
808c34bb-4e64-4eb5-9c3e-2a1e58195074    2
207f72f7-1453-40dc-919e-3551f7041b22    2
92d7aeea-7abf-47a4-bf76-a4fe0fac65c6    2
94a7ae52-fed9-4997-b972-1761aa4b89ca    2
23cbd274-ded3-43ea-9174-b5615eac3244    2
a21faf4f-3628-4e24-ac64-8471d9d12560    2
ba818380-4001-46a2-ac04-d5f322cdf3b9    2
e0d038e9-0926-4c05-8317-ec7075a04b12    2
676946dc-1b38-4a09-9bbe-9ce638cfed52    2
97ff14e8-6d08-4214-ad3b-847fa3a0e84a    2
2437bb57-1d09-4892-8c26-a60661dae1d9    2
82c2aafa-ef04-49f0-a172-0dfa8a15582e    2
7ed61f45-7e2b-449b-8ead-d34b75a5e818    2
b604ffa7-8cee-4724-bb82-f1f4553a192f    2
be79d28c-767d-4a0d-b168-e86a5842004f    2
d28bc34e-73a8-454d-9e55-dea7bdd40bee    2
4630e9ce-88ec-4159-9bc9-25ff5f8216cc    2
240b1f55-0f4e-448d-a4ce-3ecfec7a6c9d    2
ab42234a-7930-4636-8af5-3fc11ca8b34d    1
a0d95805-48db-467e-8dd5-db761eb5ae

In [99]:
pyre_df = pyre_df.set_index('Name')
pyre_df.head()

,AcresBurned,AirTankers,ArchiveYear,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,Location,MajorIncident,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders
Name,,,,,,,,,,,,,,,,,,,,,
Rim Fire,257314.0,NaN,2013,Tuolumne,55,NaN,NaN,NaN,9/6/13,20,...,3 miles east of Groveland along Hwy 120,False,NaN,8/17/13,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,NaN
Powerhouse Fire,30274.0,NaN,2013,Los Angeles,19,NaN,NaN,NaN,6/8/13,9,...,Angeles National Forest,False,NaN,5/30/13,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,NaN
Mountain Fire,27531.0,NaN,2013,Riverside,33,NaN,NaN,NaN,7/30/13,15,...,Hwy 243 & Hwy 74 near Mountain Center,False,NaN,7/15/13,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,NaN
American Fire,27440.0,NaN,2013,Placer,31,NaN,NaN,NaN,8/30/13,20,...,"Deadwood Ridge, northeast of Foresthill",False,NaN,8/10/13,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,NaN
Springs Fire,24251.0,NaN,2013,Ventura,56,47.0,8.0,117.0,5/11/13,9,...,Southbound Highway 101 at Camarillo Springs Ro...,True,2167.0,5/2/13,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,11.0


In [100]:
# unique_id2 = pyre_df.groupby(["UniqueId"]).count()["Name"]
# unique_id2

In [101]:
major = pyre_df.loc[pyre_df["MajorIncident"]==True]
major.head()



,AcresBurned,AirTankers,ArchiveYear,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,Location,MajorIncident,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders
Name,,,,,,,,,,,,,,,,,,,,,
Springs Fire,24251.0,NaN,2013,Ventura,56,47.0,8.0,117.0,5/11/13,9,...,Southbound Highway 101 at Camarillo Springs Ro...,True,2167.0,5/2/13,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,11.0
Silver Fire,20292.0,NaN,2013,Riverside,33,63.0,20.0,201.0,8/12/13,5,...,"Poppet Flats Rd near Hwy 243, south of Banning",True,2106.0,8/7/13,8.0,40.0,NaN,NaN,c400203b-a7fd-4bd8-803a-f3c74bc32a2b,20.0
Deer Fire,11429.0,NaN,2013,Tehama,52,30.0,3.0,36.0,8/29/13,6,...,"Near Deer Creek, 12 miles east of Los Molinos.",True,898.0,8/23/13,NaN,NaN,NaN,NaN,956dbcf6-db40-4b61-b235-4ede14738f1e,8.0
Clover Fire,8073.0,NaN,2013,Shasta,45,12.0,3.0,30.0,9/15/13,6,...,"Community of Igo, 10 miles southwest of Redding",True,342.0,9/9/13,10.0,201.0,NaN,NaN,92af9783-eda9-4186-afe9-447f61a92636,NaN
Chariot Fire,7055.0,NaN,2013,San Diego,37,56.0,24.0,183.0,7/15/13,9,...,"off Sunrise Hwy, 9 miles southeast of Julian",True,2147.0,7/6/13,9.0,149.0,NaN,NaN,ee19b2ec-a96a-4738-994e-fb3ea016e053,24.0


In [102]:
major["MajorIncident"].count()

383

In [103]:
major.count()

AcresBurned             383
AirTankers               26
ArchiveYear             383
Counties                383
CountyIds               383
CrewsInvolved           167
Dozers                  119
Engines                 185
Extinguished            378
Days Burned             383
Fatalities               21
Helicopters              83
Injuries                119
Latitude                383
Location                383
MajorIncident           383
PersonnelInvolved       191
Started                 383
StructuresDamaged        66
StructuresDestroyed     160
StructuresEvacuated       0
StructuresThreatened     30
UniqueId                383
WaterTenders            141
dtype: int64

In [104]:
years = pyre_df.groupby(["ArchiveYear"]).count()["UniqueId"]
years

ArchiveYear
2013    162
2014    117
2015    152
2016    185
2017    438
2018    316
2019    266
Name: UniqueId, dtype: int64

In [105]:
sample = pd.DataFrame({"Number of Fires per Year": years})
sample

,Number of Fires per Year
ArchiveYear,
2013,162
2014,117
2015,152
2016,185
2017,438
2018,316
2019,266


In [106]:
# make graph


In [107]:
# ALESSANDRO'S EVALUATION

In [108]:
mean_by_county = major.groupby(["Counties"])
mean_by_county = mean_by_county.mean()
mean_by_county.head()

,AcresBurned,AirTankers,ArchiveYear,CrewsInvolved,Dozers,Engines,Days Burned,Fatalities,Helicopters,Injuries,Latitude,MajorIncident,PersonnelInvolved,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,WaterTenders
Counties,,,,,,,,,,,,,,,,,,
Alameda,1670.000000,NaN,2016.500000,NaN,NaN,5.000000,91.500000,NaN,NaN,NaN,37.570480,True,18.000000,NaN,NaN,NaN,NaN,1.0
Amador,716.250000,4.0,2016.250000,2.5000,1.750000,10.333333,24.750000,NaN,1.500000,1.500000,28.796932,True,78.333333,NaN,34.000000,NaN,NaN,3.2
Butte,6531.107143,2.8,2015.821429,8.4375,4.687500,21.944444,57.035714,85.0,3.555556,2.333333,33.480530,True,289.095238,15.75,1724.272727,NaN,0.0,5.5
Calaveras,122.100000,NaN,2015.900000,18.0000,18.000000,17.500000,65.700000,NaN,18.000000,1.333333,34.317558,True,94.500000,1.00,2.250000,NaN,NaN,18.0
Colusa,76552.666667,NaN,2017.166667,21.0000,27.333333,91.000000,114.333333,1.0,7.666667,3.000000,39.018703,True,1080.333333,NaN,140.500000,NaN,1037.5,28.0


In [109]:
mean_by_county = mean_by_county.sort_values("AcresBurned", ascending = 0).round(0)
mean_by_county.head()

,AcresBurned,AirTankers,ArchiveYear,CrewsInvolved,Dozers,Engines,Days Burned,Fatalities,Helicopters,Injuries,Latitude,MajorIncident,PersonnelInvolved,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,WaterTenders
Counties,,,,,,,,,,,,,,,,,,
Glenn,137570.0,NaN,2016.0,11.0,5.0,6.0,125.0,1.0,5.0,3.0,39.0,True,221.0,NaN,124.0,NaN,1050.0,5.0
Colusa,76553.0,NaN,2017.0,21.0,27.0,91.0,114.0,1.0,8.0,3.0,39.0,True,1080.0,NaN,140.0,NaN,1038.0,28.0
Ventura,60281.0,NaN,2017.0,31.0,4.0,65.0,137.0,3.0,6.0,5.0,25.0,True,1213.0,158.0,545.0,NaN,NaN,9.0
Santa Barbara,58022.0,NaN,2017.0,10.0,1.0,10.0,208.0,NaN,1.0,NaN,28.0,True,548.0,280.0,544.0,NaN,1200.0,3.0
Trinity,57665.0,NaN,2016.0,44.0,48.0,7.0,91.0,3.0,NaN,1.0,41.0,True,251.0,61.0,808.0,NaN,NaN,31.0


In [119]:
mean_by_county_sum = mean_by_county.sum()
mean_by_county_sum

AcresBurned             738759.0
AirTankers                  44.0
ArchiveYear              98794.0
CrewsInvolved              487.0
Dozers                     312.0
Engines                    967.0
Days Burned             -12760.0
Fatalities                 140.0
Helicopters                173.0
Injuries                   130.0
Latitude                  1597.0
MajorIncident               49.0
PersonnelInvolved        13920.0
StructuresDamaged         1680.0
StructuresDestroyed       7788.0
StructuresEvacuated          0.0
StructuresThreatened     12079.0
WaterTenders               325.0
dtype: float64

In [110]:
# Generate a summary statistics table of mean, median, variance, standard deviation, 
# and SEM of the tumor volume for each regimen
ac_burn_mean = major.groupby('Counties')['AcresBurned'].mean()
ac_burn_median = major.groupby('Counties')['AcresBurned'].median()
ac_burn_variance = major.groupby('Counties')['AcresBurned'].var()
ac_burn_standard_dv = major.groupby('Counties')['AcresBurned'].std()
ac_burn_sem = major.groupby('Counties')['AcresBurned'].sem()

In [112]:
ac_burn_summary_stat = pd.DataFrame({"Acres Burned Mean": ac_burn_mean, 
                                      "Acres Burned Median": ac_burn_median, 
                                      "Acres Burned Variance": ac_burn_variance, 
                                      "Acres Burned Standard Deviation": ac_burn_standard_dv,
                                      "Acres Burned SEM": ac_burn_sem}
                                      )

In [113]:
ac_burn_summary_stat.round(0).head()

,Acres Burned Mean,Acres Burned Median,Acres Burned Variance,Acres Burned Standard Deviation,Acres Burned SEM
Counties,,,,,
Alameda,1670.0,1670.0,2.121800e+06,1457.0,1030.0
Amador,716.0,128.0,2.103914e+06,1450.0,513.0
Butte,6531.0,224.0,8.323339e+08,28850.0,5452.0
Calaveras,122.0,84.0,1.404000e+04,118.0,37.0
Colusa,76553.0,72.0,2.709956e+10,164619.0,67206.0


In [115]:
fires_by_county = major.set_index('Counties').reset_index()
fires_by_county.head()

,Counties,AcresBurned,AirTankers,ArchiveYear,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,Location,MajorIncident,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders
0,Ventura,24251.0,NaN,2013,56,47.0,8.0,117.0,5/11/13,9,...,Southbound Highway 101 at Camarillo Springs Ro...,True,2167.0,5/2/13,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,11.0
1,Riverside,20292.0,NaN,2013,33,63.0,20.0,201.0,8/12/13,5,...,"Poppet Flats Rd near Hwy 243, south of Banning",True,2106.0,8/7/13,8.0,40.0,NaN,NaN,c400203b-a7fd-4bd8-803a-f3c74bc32a2b,20.0
2,Tehama,11429.0,NaN,2013,52,30.0,3.0,36.0,8/29/13,6,...,"Near Deer Creek, 12 miles east of Los Molinos.",True,898.0,8/23/13,NaN,NaN,NaN,NaN,956dbcf6-db40-4b61-b235-4ede14738f1e,8.0
3,Shasta,8073.0,NaN,2013,45,12.0,3.0,30.0,9/15/13,6,...,"Community of Igo, 10 miles southwest of Redding",True,342.0,9/9/13,10.0,201.0,NaN,NaN,92af9783-eda9-4186-afe9-447f61a92636,NaN
4,San Diego,7055.0,NaN,2013,37,56.0,24.0,183.0,7/15/13,9,...,"off Sunrise Hwy, 9 miles southeast of Julian",True,2147.0,7/6/13,9.0,149.0,NaN,NaN,ee19b2ec-a96a-4738-994e-fb3ea016e053,24.0


In [116]:
clean = fires_by_county[(fires_by_county['ArchiveYear'] >= 2020) & (fires_by_county['ArchiveYear'] <= 2010)].index
fires_by_county.drop(clean, inplace=True)
fires_by_county.head()

,Counties,AcresBurned,AirTankers,ArchiveYear,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,Location,MajorIncident,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders
0,Ventura,24251.0,NaN,2013,56,47.0,8.0,117.0,5/11/13,9,...,Southbound Highway 101 at Camarillo Springs Ro...,True,2167.0,5/2/13,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,11.0
1,Riverside,20292.0,NaN,2013,33,63.0,20.0,201.0,8/12/13,5,...,"Poppet Flats Rd near Hwy 243, south of Banning",True,2106.0,8/7/13,8.0,40.0,NaN,NaN,c400203b-a7fd-4bd8-803a-f3c74bc32a2b,20.0
2,Tehama,11429.0,NaN,2013,52,30.0,3.0,36.0,8/29/13,6,...,"Near Deer Creek, 12 miles east of Los Molinos.",True,898.0,8/23/13,NaN,NaN,NaN,NaN,956dbcf6-db40-4b61-b235-4ede14738f1e,8.0
3,Shasta,8073.0,NaN,2013,45,12.0,3.0,30.0,9/15/13,6,...,"Community of Igo, 10 miles southwest of Redding",True,342.0,9/9/13,10.0,201.0,NaN,NaN,92af9783-eda9-4186-afe9-447f61a92636,NaN
4,San Diego,7055.0,NaN,2013,37,56.0,24.0,183.0,7/15/13,9,...,"off Sunrise Hwy, 9 miles southeast of Julian",True,2147.0,7/6/13,9.0,149.0,NaN,NaN,ee19b2ec-a96a-4738-994e-fb3ea016e053,24.0


In [117]:
fires_by_county['AcresBurned_per_Injury'] = fires_by_county['AcresBurned']/fires_by_county['Injuries']
fires_by_county.head().round(0)

,Counties,AcresBurned,AirTankers,ArchiveYear,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Days Burned,...,MajorIncident,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders,AcresBurned_per_Injury
0,Ventura,24251.0,NaN,2013,56,47.0,8.0,117.0,5/11/13,9,...,True,2167.0,5/2/13,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,11.0,2425.0
1,Riverside,20292.0,NaN,2013,33,63.0,20.0,201.0,8/12/13,5,...,True,2106.0,8/7/13,8.0,40.0,NaN,NaN,c400203b-a7fd-4bd8-803a-f3c74bc32a2b,20.0,780.0
2,Tehama,11429.0,NaN,2013,52,30.0,3.0,36.0,8/29/13,6,...,True,898.0,8/23/13,NaN,NaN,NaN,NaN,956dbcf6-db40-4b61-b235-4ede14738f1e,8.0,2286.0
3,Shasta,8073.0,NaN,2013,45,12.0,3.0,30.0,9/15/13,6,...,True,342.0,9/9/13,10.0,201.0,NaN,NaN,92af9783-eda9-4186-afe9-447f61a92636,NaN,1346.0
4,San Diego,7055.0,NaN,2013,37,56.0,24.0,183.0,7/15/13,9,...,True,2147.0,7/6/13,9.0,149.0,NaN,NaN,ee19b2ec-a96a-4738-994e-fb3ea016e053,24.0,588.0
